In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import scipy.optimize as sp
import matplotlib.pyplot as plt
%matplotlib inline
from collections import Counter
import xgboost as xgb
%pylab inline

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import roc_auc_score
from sklearn import linear_model, model_selection, metrics, tree, ensemble 


Populating the interactive namespace from numpy and matplotlib


In [2]:
data = pd.read_csv('../input/house-prices-advanced-regression-techniques/train.csv', index_col='Id')
data.SalePrice.value_counts()
data.info()
#['PoolArea'].unique()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1460 entries, 1 to 1460
Data columns (total 80 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   MSSubClass     1460 non-null   int64  
 1   MSZoning       1460 non-null   object 
 2   LotFrontage    1201 non-null   float64
 3   LotArea        1460 non-null   int64  
 4   Street         1460 non-null   object 
 5   Alley          91 non-null     object 
 6   LotShape       1460 non-null   object 
 7   LandContour    1460 non-null   object 
 8   Utilities      1460 non-null   object 
 9   LotConfig      1460 non-null   object 
 10  LandSlope      1460 non-null   object 
 11  Neighborhood   1460 non-null   object 
 12  Condition1     1460 non-null   object 
 13  Condition2     1460 non-null   object 
 14  BldgType       1460 non-null   object 
 15  HouseStyle     1460 non-null   object 
 16  OverallQual    1460 non-null   int64  
 17  OverallCond    1460 non-null   int64  
 18  YearBuil

In [3]:
dataT = pd.read_csv('../input/house-prices-advanced-regression-techniques/test.csv', index_col='Id')
dataT.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1459 entries, 1461 to 2919
Data columns (total 79 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   MSSubClass     1459 non-null   int64  
 1   MSZoning       1455 non-null   object 
 2   LotFrontage    1232 non-null   float64
 3   LotArea        1459 non-null   int64  
 4   Street         1459 non-null   object 
 5   Alley          107 non-null    object 
 6   LotShape       1459 non-null   object 
 7   LandContour    1459 non-null   object 
 8   Utilities      1457 non-null   object 
 9   LotConfig      1459 non-null   object 
 10  LandSlope      1459 non-null   object 
 11  Neighborhood   1459 non-null   object 
 12  Condition1     1459 non-null   object 
 13  Condition2     1459 non-null   object 
 14  BldgType       1459 non-null   object 
 15  HouseStyle     1459 non-null   object 
 16  OverallQual    1459 non-null   int64  
 17  OverallCond    1459 non-null   int64  
 18  YearB

# Let's select columns with qualitative and quantitative features

In [4]:
num_data=data.select_dtypes(exclude=['object']).axes

cat_data=data.select_dtypes(include=['object']).axes

print (cat_data)
print (num_data)

[Int64Index([   1,    2,    3,    4,    5,    6,    7,    8,    9,   10,
            ...
            1451, 1452, 1453, 1454, 1455, 1456, 1457, 1458, 1459, 1460],
           dtype='int64', name='Id', length=1460), Index(['MSZoning', 'Street', 'Alley', 'LotShape', 'LandContour', 'Utilities',
       'LotConfig', 'LandSlope', 'Neighborhood', 'Condition1', 'Condition2',
       'BldgType', 'HouseStyle', 'RoofStyle', 'RoofMatl', 'Exterior1st',
       'Exterior2nd', 'MasVnrType', 'ExterQual', 'ExterCond', 'Foundation',
       'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2',
       'Heating', 'HeatingQC', 'CentralAir', 'Electrical', 'KitchenQual',
       'Functional', 'FireplaceQu', 'GarageType', 'GarageFinish', 'GarageQual',
       'GarageCond', 'PavedDrive', 'PoolQC', 'Fence', 'MiscFeature',
       'SaleType', 'SaleCondition'],
      dtype='object')]
[Int64Index([   1,    2,    3,    4,    5,    6,    7,    8,    9,   10,
            ...
            1451, 1452, 1453, 14

In [5]:
num_feat=['MSSubClass', 'LotFrontage', 'LotArea', 'OverallQual', 'OverallCond',
        'YearBuilt', 'YearRemodAdd', 'MasVnrArea', 'BsmtFinSF1', 'BsmtFinSF2',
        'BsmtUnfSF', 'TotalBsmtSF', '1stFlrSF', '2ndFlrSF', 'LowQualFinSF',
        'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath',
        'BedroomAbvGr', 'KitchenAbvGr', 'TotRmsAbvGrd', 'Fireplaces',
        'GarageYrBlt', 'GarageCars', 'GarageArea', 'WoodDeckSF', 'OpenPorchSF',
        'EnclosedPorch', '3SsnPorch', 'ScreenPorch', 'PoolArea', 'MiscVal',
        'MoSold', 'YrSold']
cat_feat=['MSZoning', 'Street', 'Alley', 'LotShape', 'LandContour', 'Utilities',
        'LotConfig', 'LandSlope', 'Neighborhood', 'Condition1', 'Condition2',
        'BldgType', 'HouseStyle', 'RoofStyle', 'RoofMatl', 'Exterior1st',
        'Exterior2nd', 'MasVnrType', 'ExterQual', 'ExterCond', 'Foundation',
        'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2',
        'Heating', 'HeatingQC', 'CentralAir', 'Electrical', 'KitchenQual',
        'Functional', 'FireplaceQu', 'GarageType', 'GarageFinish', 'GarageQual',
        'GarageCond', 'PavedDrive', 'PoolQC', 'Fence', 'MiscFeature',
        'SaleType', 'SaleCondition']

# Detect and delete outliers

In [6]:


def detect_outliers(df,n,features):
    """
    Takes a dataframe df of features and returns a list of the indices
    corresponding to the observations containing more than n outliers according
    to the Tukey method.
    """
    outlier_indices = []
    
    # iterate over features(columns)
    for col in features:
        # 1st quartile (25%)
        Q1 = np.percentile(df[col], 25)
        # 3rd quartile (75%)
        Q3 = np.percentile(df[col],75)
        # Interquartile range (IQR)
        IQR = Q3 - Q1
        
        # outlier step
        outlier_step = 1.5 * IQR
        
        # Determine a list of indices of outliers for feature col
        outlier_list_col = df[(df[col] < Q1 - outlier_step) | (df[col] > Q3 + outlier_step )].index
        
        # append the found outlier indices for col to the list of outlier indices 
        outlier_indices.extend(outlier_list_col)
        
    # select observations containing more than 2 outliers
    outlier_indices = Counter(outlier_indices)        
    multiple_outliers = list( k for k, v in outlier_indices.items() if v > n )
    
    return multiple_outliers   

# detect outliers from Age, SibSp , Parch and Fare
Outliers_to_drop = detect_outliers(data,2,num_feat)

In [7]:
data.loc[Outliers_to_drop] # Show the outliers rows

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
Id,,,,,,,,,,,,,,,,,,,,,
49,190,RM,33.0,4456,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,NaN,NaN,NaN,0,6,2009,New,Partial,113000
94,190,C (all),60.0,7200,Pave,NaN,Reg,Lvl,AllPub,Corner,...,0,NaN,NaN,NaN,0,11,2007,WD,Normal,133900
126,190,RM,60.0,6780,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,NaN,NaN,NaN,0,6,2006,WD,Normal,84500
166,190,RL,62.0,10106,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,127500
173,160,RL,44.0,5306,Pave,NaN,IR1,Lvl,AllPub,Inside,...,0,NaN,NaN,NaN,0,6,2006,WD,Normal,239000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1351,90,RL,91.0,11643,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,NaN,NaN,NaN,0,8,2009,WD,Normal,200000
635,90,RL,64.0,6979,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,NaN,GdPrv,Shed,600,6,2010,WD,Normal,144000
911,90,RL,80.0,11600,Pave,NaN,Reg,Lvl,AllPub,Corner,...,0,NaN,NaN,NaN,0,1,2010,WD,Normal,154300


### Hoe we can see that there are only 122 lines of outliers in dataset.  This is not a significant part, we can delete these lines in a very short way.


In [8]:
data = data.drop(Outliers_to_drop, axis = 0).reset_index(drop=True)
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1338 entries, 0 to 1337
Data columns (total 80 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   MSSubClass     1338 non-null   int64  
 1   MSZoning       1338 non-null   object 
 2   LotFrontage    1109 non-null   float64
 3   LotArea        1338 non-null   int64  
 4   Street         1338 non-null   object 
 5   Alley          79 non-null     object 
 6   LotShape       1338 non-null   object 
 7   LandContour    1338 non-null   object 
 8   Utilities      1338 non-null   object 
 9   LotConfig      1338 non-null   object 
 10  LandSlope      1338 non-null   object 
 11  Neighborhood   1338 non-null   object 
 12  Condition1     1338 non-null   object 
 13  Condition2     1338 non-null   object 
 14  BldgType       1338 non-null   object 
 15  HouseStyle     1338 non-null   object 
 16  OverallQual    1338 non-null   int64  
 17  OverallCond    1338 non-null   int64  
 18  YearBuil

# Preparing the dataset for modeling
### Divide train-data and train-labels, concat train and test dataset to fill in missing values and coding categorical features

In [9]:
y_train = data["SalePrice"]
del data["SalePrice"]

Data=pd.concat([data, dataT])
Data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2797 entries, 0 to 2919
Data columns (total 79 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   MSSubClass     2797 non-null   int64  
 1   MSZoning       2793 non-null   object 
 2   LotFrontage    2341 non-null   float64
 3   LotArea        2797 non-null   int64  
 4   Street         2797 non-null   object 
 5   Alley          186 non-null    object 
 6   LotShape       2797 non-null   object 
 7   LandContour    2797 non-null   object 
 8   Utilities      2795 non-null   object 
 9   LotConfig      2797 non-null   object 
 10  LandSlope      2797 non-null   object 
 11  Neighborhood   2797 non-null   object 
 12  Condition1     2797 non-null   object 
 13  Condition2     2797 non-null   object 
 14  BldgType       2797 non-null   object 
 15  HouseStyle     2797 non-null   object 
 16  OverallQual    2797 non-null   int64  
 17  OverallCond    2797 non-null   int64  
 18  YearBuil

### Fill in the missing values with the median for quantitative features and the most frequently mentioned for qualitative:



In [10]:
for i in num_feat:
    Data[i].fillna(float(Data[i].median(axis=0)), inplace = True)
for i in cat_feat:
    Data[i].fillna(Data[i].mode()[0], inplace = True)
Data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2797 entries, 0 to 2919
Data columns (total 79 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   MSSubClass     2797 non-null   int64  
 1   MSZoning       2797 non-null   object 
 2   LotFrontage    2797 non-null   float64
 3   LotArea        2797 non-null   int64  
 4   Street         2797 non-null   object 
 5   Alley          2797 non-null   object 
 6   LotShape       2797 non-null   object 
 7   LandContour    2797 non-null   object 
 8   Utilities      2797 non-null   object 
 9   LotConfig      2797 non-null   object 
 10  LandSlope      2797 non-null   object 
 11  Neighborhood   2797 non-null   object 
 12  Condition1     2797 non-null   object 
 13  Condition2     2797 non-null   object 
 14  BldgType       2797 non-null   object 
 15  HouseStyle     2797 non-null   object 
 16  OverallQual    2797 non-null   int64  
 17  OverallCond    2797 non-null   int64  
 18  YearBuil

In [11]:
Data=pd.get_dummies(Data)

### Divide the dataset into train and test

In [12]:
X_train = Data[:1338]
X_test = Data[1338:]

print((X_train.info()))
print((X_test.info()))

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1338 entries, 0 to 1337
Columns: 284 entries, MSSubClass to SaleCondition_Partial
dtypes: float64(11), int64(25), uint8(248)
memory usage: 710.8 KB
None
<class 'pandas.core.frame.DataFrame'>
Int64Index: 1459 entries, 1461 to 2919
Columns: 284 entries, MSSubClass to SaleCondition_Partial
dtypes: float64(11), int64(25), uint8(248)
memory usage: 775.1 KB
None


# Modeling

# LinearRegression

In [13]:
LR = linear_model.LinearRegression().fit(X_train, y_train)
print (list( zip (data.columns, LR.coef_ )))

[('MSSubClass', -72.3447763370166), ('MSZoning', 58.714645138328144), ('LotFrontage', 1.1617854644106202), ('LotArea', 6762.340166924009), ('Street', 4865.283898592986), ('Alley', 335.16502563370756), ('LotShape', 99.86253852696943), ('LandContour', 20.88401378838313), ('Utilities', 14.279754850833768), ('LotConfig', 7.429342052167556), ('LandSlope', -3.949839754804998), ('Neighborhood', 17.759275632745357), ('Condition1', 10.670390634066734), ('Condition2', 25.541966775298988), ('BldgType', 7.8527710873840775), ('HouseStyle', 44.065084051654594), ('OverallQual', 3061.9575364075813), ('OverallCond', -1365.7155723117748), ('YearBuilt', 2166.836385633687), ('YearRemodAdd', 282.10705131143413), ('RoofStyle', -3417.6867429231634), ('RoofMatl', -10519.40658559009), ('Exterior1st', 106.17268956197167), ('Exterior2nd', 2170.805152519895), ('MasVnrType', 18.484515975713492), ('MasVnrArea', 4293.058227476177), ('ExterQual', 7.146949090547878), ('ExterCond', 19.908358705859882), ('Foundation', 2

In [14]:
LR.get_params()

{'copy_X': True, 'fit_intercept': True, 'n_jobs': None, 'normalize': False}

In [15]:
metrics.SCORERS.keys()

dict_keys(['explained_variance', 'r2', 'max_error', 'neg_median_absolute_error', 'neg_mean_absolute_error', 'neg_mean_squared_error', 'neg_mean_squared_log_error', 'neg_root_mean_squared_error', 'neg_mean_poisson_deviance', 'neg_mean_gamma_deviance', 'accuracy', 'roc_auc', 'roc_auc_ovr', 'roc_auc_ovo', 'roc_auc_ovr_weighted', 'roc_auc_ovo_weighted', 'balanced_accuracy', 'average_precision', 'neg_log_loss', 'neg_brier_score', 'adjusted_rand_score', 'homogeneity_score', 'completeness_score', 'v_measure_score', 'mutual_info_score', 'adjusted_mutual_info_score', 'normalized_mutual_info_score', 'fowlkes_mallows_score', 'precision', 'precision_macro', 'precision_micro', 'precision_samples', 'precision_weighted', 'recall', 'recall_macro', 'recall_micro', 'recall_samples', 'recall_weighted', 'f1', 'f1_macro', 'f1_micro', 'f1_samples', 'f1_weighted', 'jaccard', 'jaccard_macro', 'jaccard_micro', 'jaccard_samples', 'jaccard_weighted'])

In [16]:
estimatorLR = model_selection.cross_val_score(linear_model.LinearRegression(), X_train, y_train,  scoring='neg_mean_squared_error', cv=5)
 
print(estimatorLR)

[-3.99478592e+08 -5.11572276e+08 -5.74113243e+08 -3.58776260e+08
 -5.54475970e+08]


In [17]:
lasso = model_selection.cross_val_score(linear_model.Lasso(), X_train, y_train,  scoring='neg_root_mean_squared_error',  cv=5)
print(lasso)

/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:531: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 9540845829.35437, tolerance: 543129941.7404864
  positive)
/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:531: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 134242106126.9698, tolerance: 502988018.99407285
  positive)
/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:531: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 124245779387.87627, tolerance: 531550212.0112551
  positive)
/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:531: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 14735844368

[-19570.53244458 -22426.57110711 -23318.20798804 -18844.9737048
 -23400.25504937]


/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:531: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 108764161062.24268, tolerance: 509340228.7274272
  positive)


# RidgeClassifier

In [18]:
param_grid = {'alpha': [0.01, 0.05, 0.1, 0.5, 1, 2, 5, 10]}

estimator = linear_model.Ridge( random_state = 1)
optimizerR = GridSearchCV(estimator, param_grid,  scoring='neg_root_mean_squared_error',  cv=3)                    
optimizerR.fit(X_train, y_train)

print('score_train_opt', optimizerR.best_score_)
print('param_opt', optimizerR.best_params_)
Y=optimizerR.predict(X_test)
print(Y)

score_train_opt -20227.067239413132
param_opt {'alpha': 10}
[108723.74584211 172089.01501572 186009.46443315 ... 181499.94354285
 105499.36978032 217660.41723801]


# DecisionTree

In [19]:
param_grid = {'max_depth': list(range(1, 10)), 'min_samples_leaf': list(range(2, 10))}#, 'class_weightdict': ["balanced", "None"]}


estimator = tree.DecisionTreeRegressor(random_state = 1)
optimizerDT = GridSearchCV(estimator, param_grid,  scoring='neg_root_mean_squared_error', cv = 3)                    
optimizerDT.fit(X_train, y_train)

print('score_train_opt', optimizerDT.best_score_)
print('param_opt', optimizerDT.best_params_)


score_train_opt -30805.402892688002
param_opt {'max_depth': 9, 'min_samples_leaf': 7}


# RandomForestClassifier

In [20]:
param_grid = {'n_estimators': list(range(10, 100, 5)), 'min_weight_fraction_leaf': [0.001,  0.005, 0.01,  0.05, 0.1,  0.5] } 

estimator = ensemble.RandomForestRegressor(max_depth=5, random_state = 1)
optimizerRF = GridSearchCV(estimator, param_grid,  scoring='neg_root_mean_squared_error', cv=3)                    
optimizerRF.fit(X_train, y_train)

print('score_train_opt', optimizerRF.best_score_)
print('param_opt', optimizerRF.best_params_)


score_train_opt -25976.915677398618
param_opt {'min_weight_fraction_leaf': 0.005, 'n_estimators': 50}


# Stochastic Gradient Descent

In [21]:
regressor = linear_model.SGDRegressor(random_state = 0, max_iter=5)
print(model_selection.cross_val_score(regressor, X_train, y_train,  scoring='neg_root_mean_squared_error', cv=5))



[-1.56807126e+16 -2.70814552e+16 -4.25710932e+16 -3.94912087e+16
 -7.92052175e+16]


/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_stochastic_gradient.py:1211: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  ConvergenceWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_stochastic_gradient.py:1211: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  ConvergenceWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_stochastic_gradient.py:1211: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  ConvergenceWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_stochastic_gradient.py:1211: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  ConvergenceWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/linea

# GradientBoosting

In [22]:
param_grid = { 'learning_rate': [0.01, 0.05, 0.1, 0.5, 1, 2, 5, 10], 'n_estimators': list(range(10, 100, 5)) }
estimator = xgb.XGBRegressor(max_depth=5, random_state = 1, min_child_weight=3)
optimizerXGB = GridSearchCV(estimator, param_grid, scoring='neg_root_mean_squared_error', cv=3)                    
optimizerXGB.fit(X_train, y_train)

print('score_train_opt', optimizerXGB.best_score_)
print('param_opt', optimizerXGB.best_params_)


ValueError: Input contains NaN, infinity or a value too large for dtype('float32').

In [23]:
#Y=optimizerR.predict(X_test)
print(Y)
y = pd.read_csv('../input/house-prices-advanced-regression-techniques/sample_submission.csv', index_col='Id')
#print(y.info)
del y['SalePrice']
y['SalePrice']=Y
print(y)

[108723.74584211 172089.01501572 186009.46443315 ... 181499.94354285
 105499.36978032 217660.41723801]
          SalePrice
Id                 
1461  108723.745842
1462  172089.015016
1463  186009.464433
1464  196598.730060
1465  190646.649612
...             ...
2915   80309.400645
2916   74036.737565
2917  181499.943543
2918  105499.369780
2919  217660.417238

[1459 rows x 1 columns]


In [24]:
y.to_csv("./HousePrice_answer.csv")